In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

C:\Users\ROHAN\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Load dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
df = pd.read_csv(url, header=None, names=columns)

In [ ]:
df.head()

In [3]:
# Replace missing values marked as '?' with NaN and drop them
df.replace('?', np.nan, inplace=True)
df.dropna(inplace=True)

# Convert to appropriate data types
df = df.astype(float)

# Features and target
X = df.drop('target', axis=1)
y = df['target']

# Binarize the target variable
y = y.apply(lambda x: 1 if x > 0 else 0)


In [4]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# RandomForest model with GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)


# Best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# Predictions
y_pred = best_rf.predict(X_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters found:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}


In [8]:
# Evaluation
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

Accuracy: 0.8833333333333333
Confusion Matrix:
[[32  4]
 [ 3 21]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.89      0.90        36
           1       0.84      0.88      0.86        24

    accuracy                           0.88        60
   macro avg       0.88      0.88      0.88        60
weighted avg       0.88      0.88      0.88        60



In [9]:
# Function to predict based on new data
def predict_heart_disease(model, scaler, new_data):
    new_data_scaled = scaler.transform(new_data)
    prediction = model.predict(new_data_scaled)
    return "Disease" if prediction[0] == 1 else "No Disease"

In [10]:
# Example of providing new random data
new_data_1 = np.array([[63.0, 1.0, 1.0, 145.0, 233.0, 1.0, 2.0, 150.0, 0.0, 2.3, 3.0, 0.0, 6.0]])
prediction_result_1 = predict_heart_disease(best_rf, scaler, new_data_1)
print("Prediction for new data 1:", prediction_result_1)

# Another set of new data likely to predict disease
new_data_2 = np.array([[55.0, 1.0, 4.0, 180.0, 285.0, 0.0, 2.0, 120.0, 1.0, 3.0, 2.0, 1.0, 7.0]])
prediction_result_2 = predict_heart_disease(best_rf, scaler, new_data_2)
print("Prediction for new data 2:", prediction_result_2)

Prediction for new data 1: No Disease
Prediction for new data 2: Disease


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


#FRONT-END CODE 

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
from flask import Flask, request

# Load dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
df = pd.read_csv(url, header=None, names=columns)

# Replace missing values marked as '?' with NaN and drop them
df.replace('?', np.nan, inplace=True)
df.dropna(inplace=True)

# Convert to appropriate data types
df = df.astype(float)

# Features and target
X = df.drop('target', axis=1)
y = df['target']

# Binarize the target variable
y = y.apply(lambda x: 1 if x > 0 else 0)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# RandomForest model with GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# Save the model and scaler
joblib.dump(best_rf, 'heart_disease_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

# Flask app
app = Flask(__name__)

# Load the trained model and scaler
model = joblib.load('heart_disease_model.pkl')
scaler = joblib.load('scaler.pkl')

# Define a function to preprocess input data
def preprocess_data(data):
    data = np.array(data).reshape(1, -1)
    data_scaled = scaler.transform(data)
    return data_scaled

# Define a route for the home page
@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        try:
            # Get data from the form
            age = int(request.form['age'])
            sex = int(request.form['sex'])
            cp = int(request.form['cp'])
            trestbps = int(request.form['trestbps'])
            chol = int(request.form['chol'])
            fbs = int(request.form['fbs'])
            restecg = int(request.form['restecg'])
            thalach = int(request.form['thalach'])
            exang = int(request.form['exang'])
            oldpeak = float(request.form['oldpeak'])
            slope = int(request.form['slope'])
            ca = int(request.form['ca'])
            thal = int(request.form['thal'])
            
            # Preprocess the data
            data = [age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal]
            data_scaled = preprocess_data(data)
            
            # Make prediction
            prediction = model.predict(data_scaled)[0]
            if prediction == 1:
                result = 'Positive for Heart Disease'
            else:
                result = 'Negative for Heart Disease'
            
            return f'''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Prediction Result</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                background-color: #f8f9fa;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                margin: 0;
            }}
            .container {{
                background-color: #ffffff;
                padding: 20px;
                border-radius: 8px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                max-width: 500px;
                width: 100%;
                text-align: center;
            }}
            h1 {{
                color: #343a40;
            }}
            p {{
                font-size: 18px;
                color: #495057;
            }}
            a {{
                display: inline-block;
                margin-top: 20px;
                padding: 10px 20px;
                text-decoration: none;
                color: #ffffff;
                background-color: #007bff;
                border-radius: 4px;
            }}
            a:hover {{
                background-color: #0056b3;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Prediction Result</h1>
            <p>{result}</p>
            <a href="/">Go back</a>
        </div>
    </body>
    </html>
'''

        
        except Exception as e:
            return str(e)
    else:
        return'''
            <!DOCTYPE html>
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <title>Heart Disease Prediction</title>
                <style>
                    body {
                        font-family: Arial, sans-serif;
                        background-color: #f8f9fa;
                        display: flex;
                        justify-content: center;
                        align-items: center;
                        height: 100vh;
                        margin: 0;
                    }
                    .container {
                        background-color: #ffffff;
                        padding: 20px;
                        border-radius: 8px;
                        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                        max-width: 500px;
                        width: 100%;
                    }
                    h1 {
                        text-align: center;
                        color: #343a40;
                    }
                    form {
                        display: flex;
                        flex-direction: column;
                    }
                    label {
                        margin-top: 10px;
                        font-weight: bold;
                        color: #495057;
                    }
                    input {
                        padding: 10px;
                        margin-top: 5px;
                        border: 1px solid #ced4da;
                        border-radius: 4px;
                        font-size: 16px;
                    }
                    button {
                        margin-top: 20px;
                        padding: 10px;
                        border: none;
                        border-radius: 4px;
                        background-color: #007bff;
                        color: #ffffff;
                        font-size: 16px;
                        cursor: pointer;
                    }
                    button:hover {
                        background-color: #0056b3;
                    }
                </style>
            </head>
            <body>
                <div class="container">
                    <h1>Heart Disease Prediction</h1>
                    <form action="/" method="post">
                        <label for="age">Age:</label>
                        <input type="number" id="age" name="age" required>
                        <label for="sex">Sex (1 = male, 0 = female):</label>
                        <input type="number" id="sex" name="sex" required>
                        <label for="cp">Chest Pain Type (0-3):</label>
                        <input type="number" id="cp" name="cp" required>
                        <label for="trestbps">Resting Blood Pressure (in mm Hg):</label>
                        <input type="number" id="trestbps" name="trestbps" required>
                        <label for="chol">Serum Cholestoral (in mg/dl):</label>
                        <input type="number" id="chol" name="chol" required>
                        <label for="fbs">Fasting Blood Sugar > 120 mg/dl (1 = true; 0 = false):</label>
                        <input type="number" id="fbs" name="fbs" required>
                        <label for="restecg">Resting Electrocardiographic Results (0-2):</label>
                        <input type="number" id="restecg" name="restecg" required>
                        <label for="thalach">Maximum Heart Rate Achieved:</label>
                        <input type="number" id="thalach" name="thalach" required>
                        <label for="exang">Exercise Induced Angina (1 = yes; 0 = no):</label>
                        <input type="number" id="exang" name="exang" required>
                        <label for="oldpeak">ST Depression Induced by Exercise Relative to Rest:</label>
                        <input type="number" step="0.1" id="oldpeak" name="oldpeak" required>
                        <label for="slope">Slope of the Peak Exercise ST Segment (0-2):</label>
                        <input type="number" id="slope" name="slope" required>
                        <label for="ca">Number of Major Vessels (0-3) Colored by Flourosopy:</label>
                        <input type="number" id="ca" name="ca" required>
                        <label for="thal">Thalassemia (1 = normal; 2 = fixed defect; 3 = reversable defect):</label>
                        <input type="number" id="thal" name="thal" required>
                        <button type="submit">Predict</button>
                    </form>
                </div>
            </body>
            </html>
        '''


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


C:\Users\ROHAN\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters found:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jun/2024 18:52:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2024 18:52:30] "GET /favicon.ico HTTP/1.1" 404 -
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [03/Jun/2024 18:53:11] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2024 18:53:15] "GET / HTTP/1.1" 200 -


In [1]:
####################################

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from flask import Flask, request, render_template_string

# Load dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
df = pd.read_csv(url, header=None, names=columns)

# Replace missing values marked as '?' with NaN and drop them
df.replace('?', np.nan, inplace=True)
df.dropna(inplace=True)

# Convert to appropriate data types
df = df.astype(float)

# Features and target
X = df.drop('target', axis=1)
y = df['target']

# Binarize the target variable
y = y.apply(lambda x: 1 if x > 0 else 0)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# RandomForest model with GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train, y_train)

# Flask app
app = Flask(__name__)

# Define a function to preprocess input data
def preprocess_data(data):
    data = np.array(data).reshape(1, -1)
    data_scaled = scaler.transform(data)
    return data_scaled

# Define a route for the home page
@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        try:
            # Get data from the form
            age = int(request.form['age'])
            sex = int(request.form['sex'])
            cp = int(request.form['cp'])
            trestbps = int(request.form['trestbps'])
            chol = int(request.form['chol'])
            fbs = int(request.form['fbs'])
            restecg = int(request.form['restecg'])
            thalach = int(request.form['thalach'])
            exang = int(request.form['exang'])
            oldpeak = float(request.form['oldpeak'])
            slope = int(request.form['slope'])
            ca = int(request.form['ca'])
            thal = int(request.form['thal'])
            
            # Preprocess the data
            data = [age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal]
            data_scaled = preprocess_data(data)
            
            # Make prediction
            prediction = best_rf.predict(data_scaled)[0]
            if prediction == 1:
                result = 'Positive for Heart Disease'
            else:
                result = 'Negative for Heart Disease'
            
            return render_template_string(html_code, prediction_text=result)
        
        except Exception as e:
            return f"An error occurred: {e}"
    
    return render_template_string(html_code, prediction_text="")

html_code = """
<html lang="en">  
<head>  
   <meta charset="UTF-8">  
   <meta name="viewport" content="width=device-width, initial-scale=1.0">  
   <meta http-equiv="X-UA-Compatible" content="ie=edge">  
   <title>Heart Disease Prediction</title>  
</head>  
<style>  
   @import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@500&display=swap');  
 *{  
   margin: 0;  
   padding: 0;  
   box-sizing: border-box;  
   font-family: 'Montserrat', sans-serif;  
   color:black;  
 }  
 body{  
   background: url("https://cdn.pixabay.com/photo/2016/08/10/20/26/stethoscope-1584223_1280.jpg") no-repeat top center;  
   padding: 0 10px;  
   background-size: cover;  
 }  
 .wrapper{  
   max-width: 500px;  
   width: 100%;  
   background: rgba(0, 0, 0, 0.7);  
   margin: 20px auto;  
   padding: 30px;  
   box-shadow: 1px 1px 2px rgba(0, 0, 0, 1.25);  
 }  
 .wrapper .title{  
   font-size: 24px;  
   font-weight: 700;  
   margin-bottom: 25px;  
   color: #fec107;  
   text-transform: uppercase;  
   text-align: center;  
 }  
 .wrapper .form{  
   width: 100%;  
 }  
 .wrapper .form .input_field{  
   margin-bottom: 15px;  
   display: flex;  
   align-items: center;  
 }  
 .wrapper .form .input_field label{  
   width:80px;  
   font: bold;  
   color: wheat;  
   margin-right: 10px;  
   font-size: 14px;  
 }  
 .wrapper .form .input_field input{  
   width: 100px;  
 }  
 .wrapper .form .input_field .textarea{  
   resize: none;  
   height: 250px;  
   width: 500px;  
 }  
 </style>  
 <body>  
 <form action="/" method="post">  
   <div class="wrapper">  
     <div class="title">  
       <h1 style="color: white;">Heart Disease Prediction</h1>  
     </div>  
     <div class="form">  
       <div class="input_field">  
         <textarea style="color: white; background: rgba(0, 0, 0, 0.3);" class="textarea" readonly>  
 It's a clean, easy to understand set of data. However, the meaning of some of the column headers are not obvious. Here's what they mean,  
 Age: displays the age of the individual.  
 Sex: displays the gender of the individual using the following format :  
 1 = male  
 0 = female  
 Chest-pain type: displays the type of chest-pain experienced by the individual using the following format :  
 0 = typical angina  
 1 = atypical angina  
 2 = non — anginal pain  
 3 = asymptotic  
 Resting Blood Pressure: displays the resting blood pressure value of an individual in mmHg (unit)  
 Serum Cholestrol: displays the serum cholesterol in mg/dl (unit)  
 Fasting Blood Sugar: compares the fasting blood sugar value of an individual with 120mg/dl.  
 If fasting blood sugar > 120mg/dl then : 1 (true) else : 0 (false)  
 Resting ECG : displays resting electrocardiographic results  
 0 = normal  
 1 = having ST-T wave abnormality  
 2 = left ventricular hyperthrophy  
 Max heart rate achieved : displays the max heart rate achieved by an individual.  
 Exercise induced angina :  
 1 = yes  
 0 = no  
 ST depression induced by exercise relative to rest: displays the value which is an integer or float.  
 Peak exercise ST segment :  
 1 = upsloping  
 2 = flat  
 3 = downsloping  
 Number of major vessels (0–3) colored by flourosopy : displays the value as integer or float.  
 Thal : displays the thalassemia :  
 0 = normal  
 1 = fixed defect  
 2 = reversible defect  
 Diagnosis of heart disease : Displays whether the individual is suffering from heart disease or not :  
 0 = absence  
 1, 2, 3, 4 = present.  
           </textarea>  
        </div>  
       <div class="input_field">  
         <label>AGE</label>  
         <input type="number" id="age" min="0" max="150" name="age" class="input_text">  
         <br/>  <i style="font-size: 10px; color: white;">(Age: 1 - 150)</i>  
       </div>  
       <div class="input_field">  
         <label>SEX</label>  
          <input type="number" min="0" max="1" name="sex" id="sex" class="input_text">  
          <br/>  <i style="font-size: 10px; color: white;">(Male: 1 & Female: 0)</i>  
        </div>  
        <div class="input_field">  
         <label>CP</label>  
          <input type="number" name="cp" id="cp" min="0" max="3" class="input_text">  
          <br/>  <i style="font-size: 10px; color: white;">(Enter Single Value From Range 0-3)</i>  
        </div>  
        <div class="input_field">  
         <label>TRESTBPS</label>  
          <input name="trestbps" id="trestbps" type="number" min="0" step="1" class="input_text">  
          <br/>  <i style="font-size: 10px; color: white;">(Enter Non-Decimal Value)</i>  
        </div>  
        <div class="input_field">  
         <label>CHOL</label>  
          <input name="chol" id="chol" type="number" min="0" step="1" class="input_text">  
          <br/>  <i style="font-size: 10px; color: white;">(Enter Non-Decimal Value)</i>  
        </div>  
        <div class="input_field">  
         <label>FBS</label>  
          <input type="number" min="0" max="1" name="fbs" id="fbs" class="input_text">  
          <br/>  <i style="font-size: 10px; color: white;">(1 = True; 0 = False)</i>  
        </div>  
        <div class="input_field">  
         <label>RESTECG</label>  
          <input type="number" name="restecg" min="0" max="2" id="restecg" class="input_text">  
          <br/>  <i style="font-size: 10px; color: white;">(Enter Single Value From Range 0-2)</i>  
        </div>  
        <div class="input_field">  
         <label>THALACH</label>  
          <input name="thalach" id="thalach" type="number" min="0" step="1" class="input_text">  
          <br/>  <i style="font-size: 10px; color: white;">(Enter Non-Decimal Value)</i>  
        </div>  
        <div class="input_field">  
         <label>EXANG</label>  
          <input type="number" min="0" max="1" name="exang" id="exang" class="input_text">  
          <br/>  <i style="font-size: 10px; color: white;">(Exercise: 1 = YES; 0 = NO)</i>  
        </div>  
        <div class="input_field">  
         <label>OLDPEAK</label>  
          <input name="oldpeak" id="oldpeak" type="number" min="0" step="0.01" class="input_text">  
          <br/>  <i style="font-size: 10px; color: white;">(Enter Decimal Value)</i>  
        </div>  
        <div class="input_field">  
         <label>SLOPE</label>  
          <input type="number" min="0" max="2" name="slope" id="slope" class="input_text">  
          <br/>  <i style="font-size: 10px; color: white;">(Enter Single Value From Range 0-2)</i>  
        </div>  
        <div class="input_field">  
         <label>CA</label>  
          <input type="number" name="ca" min="0" max="4" id="ca" class="input_text">  
          <br/>  <i style="font-size: 10px; color: white;">(Enter Single Value From Range 0-4)</i>  
        </div>  
        <div class="input_field">  
         <label>THAL</label>  
          <input type="number" name="thal" min="0" max="3" id="thal" class="input_text">  
          <br>  <i style="font-size: 10px; color: white;">(Enter Single Value From Range 0-3)</i>  
        </div>  
        <div class="input_field">  
         <input type="submit" style="color:black; background: skyblue;" id="submit" value="SUBMIT">  
       </div>  
       <div>  
         <center>  
           <h2 style="color: white; margin-bottom:10px;">  
             {{prediction_text}}  
           </h2>  
         </center>  
       </div>  
     </div>  
   </div>  
 </form>  
 </body>  
 </html>  
"""

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


C:\Users\ROHAN\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters found:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Jun/2024 21:57:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2024 21:58:00] "GET /favicon.ico HTTP/1.1" 404 -
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [27/Jun/2024 21:59:29] "POST / HTTP/1.1" 200 -
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [27/Jun/2024 22:01:15] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2024 22:01:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2024 22:01:47] "GET / HTTP/1.1" 200 -
